In [1]:
from typing import TypeVar
import sqlalchemy as sa
from sqlalchemy import NullPool
import pandas as pd

SelfClickHouseConnection = TypeVar("SelfClickHouseConnection", bound="ClickHouseConnection")

class ClickHouseConnection:
    """Класс контекстного менеджера для работы с ClickHouse"""
    
    def __init__(self, host: str, username: str, password: str, port: str = "9000"):
        """
        Параметры:
            host: хост ClickHouse
            username: имя пользователя ClickHouse
            password: пароль пользователя ClickHouse
            port: порт ClickHouse
        """
        
        self.session = None
        self.__host = host
        self.__port = port
        self.__username = username
        self.__password = password
        
    def __get_url(self) -> str:
        """Метод формирует connection string для подключения к ClickHouse"""
        
        url = "clickhouse+native://{username}:{password}@{host}:{port}"
        url = url.format(
            host = self.__host,
            port = self.__port,
            username = self.__username,
            password = self.__password
        )
        return url
        
    def __enter__(self: SelfClickHouseConnection) -> SelfClickHouseConnection:
        engine = sa.create_engine(self.__get_url(), poolclass=NullPool)
        self.session = engine.connect()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.session.close()
        
    def read_sql(self, query: str) -> pd.DataFrame:
        """
        Запрос к БД через фреймворк pandas
        
        Параметры:
            query: SQL в формате строки
            
        Возвращает:
            датафрейм с результатами запроса
        """
        
        with self as connector:
            return pd.read_sql(query, con=connector.session)

In [2]:
ch_host = "10.120.1.11"
ch_port = 8123
ch_username = "amarbuliev"
ch_password = "KuUNgVlPQiSN6dRqk"

connection_manager = ClickHouseConnection(host=ch_host, username=ch_username, password=ch_password)

df = connection_manager.read_sql("SELECT * FROM sandbox.professional_interests")

In [7]:
df.teacher_id

0         0
1         1
2         2
3         3
4         5
       ... 
165    4202
166    4217
167    4234
168    4269
169    4301
Name: teacher_id, Length: 170, dtype: int64

In [10]:
import json

#df.set_index('teacher_id', inplace=True)

# Convert the DataFrame to a Python dict
teacher_interests_dict = df['professional_interests'].squeeze().to_dict()

# Define the file path (in the current folder) and file name
file_name = 'teacher_interests.json'

# Write the Python dictionary to a JSON file with UTF-8 encoding
with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(teacher_interests_dict, f, ensure_ascii=False, indent=4)

print(f"File '{file_name}' has been saved successfully in the current directory.")

File 'teacher_interests.json' has been saved successfully in the current directory.
